In [ ]:
!pip install tweepy

In [ ]:
!pip install tweepy --upgrade

     |████████████████████████████████| 77 kB 2.7 MB/s 
     |████████████████████████████████| 63 kB 929 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
! pip install tweet-preprocessor

In [ ]:
!pip install spacy-langdetect

     |████████████████████████████████| 998 kB 5.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993431 sha256=c2a4cf2a5ce396016d1667a330a97d1d73219af6134fb22173ab9b07d1738a64
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect


In [ ]:
from tweepy import OAuthHandler

import nltk
import spacy
from spacy_langdetect import LanguageDetector
from nltk.corpus import stopwords
import preprocessor as p
import tweepy 
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import regex as re
import os
import time
from datetime import datetime
from tweepy import Client
from textblob import TextBlob

In [ ]:
print(tweepy.__version__)

4.9.0


In [ ]:
# Twitter credentials
# Obtain them from your twitter developer account
api_key = 'f480rt4C2qwxWM0sk1FILrRjp'
api_secret = 'Hd6aWlaZMV8w0hTHpd3NvIcRlSAUbf4TlfXkRVl7XwUGmlVZTc'
access_key = '1524266583686017025-BtwRNUFf8D9cqTwlSgwWHFiboJtov0'
access_secret = 'zFqdqTjF6iiDdB3RU0tXJexSZq0QVTlUs8Z2M8QbYRNrk'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAANLecQEAAAAAqeNIjyNnoxMUebzlIJfxUF4C8yU%3D2M3QuzL8JWkxTfR0Rzp2TPU8WAyio1vx7cBpfMU3MgkEoNr0GW'
client = tweepy.Client(bearer_token)


In [ ]:
#response = client.search_recent_tweets("elonmusk")
# The method returns a Response object, a named tuple with data, includes,
# errors, and meta fields
#print(response.meta)

# In this case, the data field of the Response returned is a list of Tweet
# objects
#tweets = response.data

# Each Tweet object has default ID and text fields
#for tweet in tweets:
    #print(tweet.id)
    #print(tweet.text)

# By default, this endpoint/method returns 10 results
# # You can retrieve up to 100 Tweets by specifying max_results
# response = client.search_recent_tweets("Tweepy", max_results=100

In [ ]:
# Define a pandas dataframe to store the date:
db_tweets = pd.DataFrame(columns = ['tweetID', 'AuthorID', 'lang','created_at', 'text'])

In [ ]:
def scraptweets(search_words, db_tweets, no_tweets):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
                            
    start_run = time.time()
    
    # Collect tweets using the Cursor object
    # .Cursor() returns an object that you can iterate or loop over to access the data collected.
    # Each item in the iterator has various attributes that you can access to get information about each tweet
    tweets = tweepy.Paginator(client.search_recent_tweets, query=search_words, 
                          tweet_fields=['id', 'created_at', 'author_id'], 
                          user_fields=['public_metrics', 'location', 'username'],
                          max_results=100).flatten(limit=500)
    
    for tweet in tweets:
      # Pull the values
      text=tweet.text
      author_id = tweet.author_id
      tweetID = tweet.id
      lang=tweet.lang
      created_at=tweet.created_at
            
      # Add the 11 variables to the empty list - ith_tweet:
      ith_tweet = [tweetID, author_id, lang, created_at, text]
      # Append to dataframe - db_tweets
      try:
        db_tweets.loc[len(db_tweets)] = ith_tweet
      except Exception as e:
        print(e)
      # increase counter - noTweets  
      no_tweets += 1
    
    # Run ended:
    end_run = time.time()
    duration_run = round((end_run-start_run)/60, 2)
    
    print('no. of tweets scraped for run 1 is {}'.format(no_tweets))
    print('time take for 1 run to complete is {} mins'.format(duration_run))
    
    return no_tweets
    

In [ ]:
num_runs=981
tweets=0
for i in range(num_runs):
  search_words = '#Hospitalstay OR #hospitalvisit OR #emergencyroomvisit OR #ERvisit OR #Hospitalexperience OR #patientexperience OR #Health OR #patient OR #trauma OR #surgery OR #critical OR #procedure OR #symptoms OR #pain OR #medicine'
  # Call the function scraptweets
  tweets=scraptweets(search_words, db_tweets, tweets)

  #new search word string
  search_words = 'Hospital stay OR hospital visit OR emergencyroom visit OR ER visit OR Hospital experience OR patient,  experience OR Health OR patient OR trauma OR surgery OR critical OR procedure OR symptoms OR pain OR medicine'
  # Call the function scraptweets
  tweets=scraptweets(search_words, db_tweets, tweets)

  #new search word string
  search_words = 'Hospital OR clinic OR urgent care OR emergency room OR ED OR Nurse OR doctor OR medical professional OR registered nurse OR dr OR patient rep OR patinet engagement OR Treatment OR treat OR assist OR care OR Surgery OR IV OR blood OR ICU OR NICU OR urgent care OR emergency room OR triage OR ED OR bill OR doctor OR bill OR health insurance OR Monitor OR heal OR recover OR care OR cure OR dying OR dead OR sicker OR sick OR ill OR illness OR condition '
  # Call the function scraptweets
  tweets=scraptweets(search_words, db_tweets, tweets)


no. of tweets scraped for run 1 is 500
time take for 1 run to complete is 0.07 mins
no. of tweets scraped for run 1 is 1000
time take for 1 run to complete is 0.07 mins
no. of tweets scraped for run 1 is 1500
time take for 1 run to complete is 0.09 mins
no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 0.09 mins
no. of tweets scraped for run 1 is 2500
time take for 1 run to complete is 0.09 mins
no. of tweets scraped for run 1 is 3000
time take for 1 run to complete is 0.21 mins
no. of tweets scraped for run 1 is 3500
time take for 1 run to complete is 0.2 mins
no. of tweets scraped for run 1 is 4000
time take for 1 run to complete is 0.17 mins
no. of tweets scraped for run 1 is 4500
time take for 1 run to complete is 0.12 mins
no. of tweets scraped for run 1 is 5000
time take for 1 run to complete is 0.12 mins
no. of tweets scraped for run 1 is 5500
time take for 1 run to complete is 0.13 mins
no. of tweets scraped for run 1 is 6000
time take for 1 run to comp

KeyboardInterrupt: ignored

In [ ]:
db_tweets

,tweetID,AuthorID,lang,created_at,text
0,1525086668348346368,1524037408047644672,None,2022-05-13 12:12:51+00:00,RT @model_svirid: #Billionaires!#Millionaires!...
1,1525086665236434945,709580995155136513,None,2022-05-13 12:12:50+00:00,RT @harrypetsanis: There is no such thing as '...
2,1525086655438327809,1151202637582348294,None,2022-05-13 12:12:48+00:00,RT @model_svirid: #Billionaires!#Millionaires!...
3,1525086632847757314,1524037408047644672,None,2022-05-13 12:12:42+00:00,#Billionaires!#Millionaires!#Sponsors!#Philant...
4,1525086553978060810,701079038,None,2022-05-13 12:12:23+00:00,RT @ANMF_MeunerieFr: 🌾Décryptage #meunerie🥖Sur...
...,...,...,...,...,...
114952,1525131356480581632,1320615780552044544,None,2022-05-13 15:10:25+00:00,RT @redjaspershopp: Just ONE day away! \nStop ...
114953,1525131355138494466,20157199,None,2022-05-13 15:10:25+00:00,"RT @GordonBrown: Tragically, we are sleepwalki..."
114954,1525131354853154816,1468522753909489664,None,2022-05-13 15:10:25+00:00,RT @ujjainumc: @ujjainumc jas made Green Net c...
114955,1525131354815479808,1001715489951842304,None,2022-05-13 15:10:25+00:00,@RockstarGames @GTASeries @_HEALTH_ so you ack...


In [ ]:
unique1= db_tweets['tweetID'].unique()
unique2=db_tweets['text'].unique()

In [ ]:
print(len(unique1), ' ',  len(unique2))

78232   47992


In [ ]:
db_tweets.drop_duplicates(subset ="tweetID",
                     keep = False, inplace = True)

In [ ]:
db_tweets

,tweetID,AuthorID,lang,created_at,text
497,1525079862410625024,1299576198591385600,None,2022-05-13 11:45:48+00:00,RT @LapsiaSnehal: It is always nice when the C...
498,1525079857201352704,1334017178,None,2022-05-13 11:45:47+00:00,"mohali: Mohali blast: Babbar Khalsa, gangsters..."
499,1525079805674274816,1189832851371347969,None,2022-05-13 11:45:34+00:00,Polycystic Kidney Disease Symptoms\n.\nhttps:/...
808,1525086602439057408,1518709954215124993,None,2022-05-13 12:12:35+00:00,"@AmoneyResists ""Since Rand Paul (R-Moscow) jus..."
809,1525086602195787776,355844222,None,2022-05-13 12:12:35+00:00,"RT @freethought202: Gareth, aged 40, is the fi..."
...,...,...,...,...,...
114952,1525131356480581632,1320615780552044544,None,2022-05-13 15:10:25+00:00,RT @redjaspershopp: Just ONE day away! \nStop ...
114953,1525131355138494466,20157199,None,2022-05-13 15:10:25+00:00,"RT @GordonBrown: Tragically, we are sleepwalki..."
114954,1525131354853154816,1468522753909489664,None,2022-05-13 15:10:25+00:00,RT @ujjainumc: @ujjainumc jas made Green Net c...
114955,1525131354815479808,1001715489951842304,None,2022-05-13 15:10:25+00:00,@RockstarGames @GTASeries @_HEALTH_ so you ack...


In [ ]:
#create a new df to store filtered tweets
db_cleanTweets=pd.DataFrame(columns = ['tweetID', 'tweetsORG','cleanTweets','Hashtags'])

In [ ]:
#data cleaning
nlp = spacy.load('en')  
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True) 

for index, row in db_tweets.iterrows():
  #detect if english
  tweet=row['text']
  doc = nlp(tweet) 
  detect_language = doc._.language 
  #skip if its not english
  if detect_language['language']!='en':
      continue
  #find all hashtags    
  hashtag=re.findall(r'#(\w+)', tweet)
  #remove hashtags, URL, emojis, mention, number, etc 
  clean=p.clean(tweet)
  ith_row=[row['tweetID'], tweet, clean, hashtag]
  db_cleanTweets.loc[len(db_cleanTweets)] = ith_row  

In [ ]:
db_cleanTweets

,tweetID,tweetsORG,cleanTweets,Hashtags
0,1525079862410625024,RT @LapsiaSnehal: It is always nice when the C...,: It is always nice when the CT and OGD correl...,"[FOAMrad, FOAMed, meded, radres, futureradres,..."
1,1525079857201352704,"mohali: Mohali blast: Babbar Khalsa, gangsters...","mohali: Mohali blast: Babbar Khalsa, gangsters...","[news, worldnews, auto, entertainment, busines..."
2,1525079805674274816,Polycystic Kidney Disease Symptoms\n.\nhttps:/...,Polycystic Kidney Disease Symptoms..,"[symptoms, polycystic, kidney, kidneydisease, ..."
3,1525086602439057408,"@AmoneyResists ""Since Rand Paul (R-Moscow) jus...","""Since Rand Paul (R-Moscow) just unilaterally ...",[]
4,1525086602195787776,"RT @freethought202: Gareth, aged 40, is the fi...",": Gareth, aged , is the first Australian to re...",[]
...,...,...,...,...
63234,1525131356480581632,RT @redjaspershopp: Just ONE day away! \nStop ...,: Just ONE day away! Stop by to support awaren...,[]
63235,1525131355138494466,"RT @GordonBrown: Tragically, we are sleepwalki...",": Tragically, we are sleepwalking into the nex...",[]
63236,1525131354853154816,RT @ujjainumc: @ujjainumc jas made Green Net c...,: jas made Green Net covering compulsory for u...,[]
63237,1525131354815479808,@RockstarGames @GTASeries @_HEALTH_ so you ack...,so you acknowledged max payne 's year annivers...,[]


In [ ]:
filename='/content/data/cleaned_tweets.csv'
db_cleanTweets.to_csv(filename, index = False, encoding='utf-8')